In [95]:
!pip install torch==2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tokenizers
!pip install transformers

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [96]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('2.0.0+cu117', '4.35.2', '0.15.0')

In [97]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from torch.nn import DataParallel
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import precision_score, recall_score, matthews_corrcoef, log_loss, hamming_loss
import numpy as np



# import matplotlib.pyplot as plt
# %matplotlib inline
# import plotly.express as px
# import seaborn as sns
print(f"Let's use {torch.cuda.device_count()} GPUs!")


Let's use 3 GPUs!


In [98]:
movies_df = pd.read_csv("wiki_movie_plots_deduped.csv")
movies_df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [99]:
movies_df = movies_df[(movies_df["Origin/Ethnicity"]=="American") | (movies_df["Origin/Ethnicity"]=="British")]
movies_df = movies_df[["Plot", "Genre"]]
drop_indices = movies_df[movies_df["Genre"] == "unknown" ].index
movies_df.drop(drop_indices, inplace=True)

# Combine genres: 1) "sci-fi" with "science fiction" &  2) "romantic comedy" with "romance"
movies_df["Genre"].replace({"sci-fi": "science fiction", "romantic comedy": "romance"}, inplace=True)

# Choosing movie genres based on their frequency
shortlisted_genres = movies_df["Genre"].value_counts().reset_index(name="count").query("count > 200")["index"].tolist()
movies_df = movies_df[movies_df["Genre"].isin(shortlisted_genres)].reset_index(drop=True)

# Shuffle DataFrame
movies_df = movies_df.sample(frac=1).reset_index(drop=True)

# Sample roughly equal number of movie plots from different genres (to reduce class imbalance issues)
movies_df = movies_df.groupby("Genre").head(400).reset_index(drop=True)

label_encoder = LabelEncoder()
movies_df["genre_encoded"] = label_encoder.fit_transform(movies_df["Genre"].tolist())

movies_df = movies_df[["Plot", "Genre", "genre_encoded"]]
movies_df

def tokenize_data(dataframe, tokenizer):
    return tokenizer(dataframe["Plot"].tolist(), padding=True, truncation=True, max_length=512, return_tensors="pt")

# Label encoding for genres
label_encoder = LabelEncoder()
movies_df["genre_encoded"] = label_encoder.fit_transform(movies_df["Genre"])


train_df, eval_df = train_test_split(movies_df, test_size=0.2, stratify=movies_df["Genre"], random_state=42)

In [100]:
import torch
from torch.nn import DataParallel
from torch.optim import AdamW
from sklearn.metrics import precision_score, recall_score, matthews_corrcoef, log_loss, hamming_loss
from torch.nn.functional import softmax
import numpy as np

# Define your custom train_and_evaluate function
def train_and_evaluate(model, train_loader, eval_loader, device, optimizer=None, criterion=None):
    if optimizer is None:
        optimizer = AdamW(model.parameters(), lr=5e-5)

    if torch.cuda.device_count() > 1:
        print(f"Let's use {torch.cuda.device_count()} GPUs!")
        model = DataParallel(model)

    model.to(device)

    # Training Loop
    for epoch in range(2):  # Adjust the number of epochs as needed
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = criterion(outputs.logits, batch['labels'])
            total_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        average_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch+1} completed, Average Training Loss: {average_train_loss:.4f}")

    # Evaluation Loop
    model.eval()
    all_predictions = []
    all_true_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in eval_loader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)

            logits = outputs.logits
            probs = softmax(logits, dim=1)
            all_probs.extend(probs.cpu().numpy())

            predictions = torch.argmax(logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_true_labels.extend(inputs['labels'].cpu().numpy())

    # Calculate Metrics
    precision = precision_score(all_true_labels, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_true_labels, all_predictions, average='weighted')
    mcc = matthews_corrcoef(all_true_labels, all_predictions)
    logloss = log_loss(all_true_labels, all_probs, labels=np.unique(all_true_labels))
    hammingloss = hamming_loss(all_true_labels, all_predictions)

    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, MCC: {mcc:.4f}, Log Loss: {logloss:.4f}, Hamming Loss: {hammingloss:.4f}")


In [101]:
from torch.utils.data import Dataset
import torch.nn as nn

class MovieDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [102]:
# BERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_train_encodings = tokenize_data(train_df, bert_tokenizer)
bert_eval_encodings = tokenize_data(eval_df, bert_tokenizer)

bert_train_dataset = MovieDataset(bert_train_encodings, train_df["genre_encoded"].tolist())
bert_eval_dataset = MovieDataset(bert_eval_encodings, eval_df["genre_encoded"].tolist())

bert_train_loader = DataLoader(bert_train_dataset, batch_size=18, shuffle=True)
bert_eval_loader = DataLoader(bert_eval_dataset, batch_size=18)

bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
bert_optimizer = AdamW(bert_model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss(reduction='mean')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [103]:
# RoBERTa Tokenizer and Model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_train_encodings = tokenize_data(train_df, roberta_tokenizer)
roberta_eval_encodings = tokenize_data(eval_df, roberta_tokenizer)

roberta_train_dataset = MovieDataset(roberta_train_encodings, train_df["genre_encoded"].tolist())
roberta_eval_dataset = MovieDataset(roberta_eval_encodings, eval_df["genre_encoded"].tolist())

roberta_train_loader = DataLoader(roberta_train_dataset, batch_size=18, shuffle=True)
roberta_eval_loader = DataLoader(roberta_eval_dataset, batch_size=18)

roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label_encoder.classes_))
roberta_optimizer = AdamW(roberta_model.parameters(), lr=5e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [104]:
# XLNet Tokenizer and Model
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_train_encodings = tokenize_data(train_df, xlnet_tokenizer)
xlnet_eval_encodings = tokenize_data(eval_df, xlnet_tokenizer)

xlnet_train_dataset = MovieDataset(xlnet_train_encodings, train_df["genre_encoded"].tolist())
xlnet_eval_dataset = MovieDataset(xlnet_eval_encodings, eval_df["genre_encoded"].tolist())

xlnet_train_loader = DataLoader(xlnet_train_dataset, batch_size=18, shuffle=True)
xlnet_eval_loader = DataLoader(xlnet_eval_dataset, batch_size=18)

xlnet_model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=len(label_encoder.classes_))
xlnet_optimizer = AdamW(xlnet_model.parameters(), lr=5e-5)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [105]:
# DistilBERT Tokenizer and Model
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_train_encodings = tokenize_data(train_df, distilbert_tokenizer)
distilbert_eval_encodings = tokenize_data(eval_df, distilbert_tokenizer)

distilbert_train_dataset = MovieDataset(distilbert_train_encodings, train_df["genre_encoded"].tolist())
distilbert_eval_dataset = MovieDataset(distilbert_eval_encodings, eval_df["genre_encoded"].tolist())

distilbert_train_loader = DataLoader(distilbert_train_dataset, batch_size=18, shuffle=True)
distilbert_eval_loader = DataLoader(distilbert_eval_dataset, batch_size=18)

distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))
distilbert_optimizer = AdamW(distilbert_model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [106]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss(reduction='mean')
# Train and Evaluate BERT
print("Training and evaluating BERT...")
train_and_evaluate(bert_model, bert_train_loader, bert_eval_loader, device,optimizer=bert_optimizer,criterion=criterion)

# Train and Evaluate RoBERTa
print("Training and evaluating RoBERTa...")
train_and_evaluate(roberta_model, roberta_train_loader, roberta_eval_loader, device,optimizer=roberta_optimizer,criterion=criterion)


# Train and Evaluate XLNet
print("Training and evaluating XLNet...")
train_and_evaluate(xlnet_model, xlnet_train_loader, xlnet_eval_loader, device,optimizer=xlnet_optimizer,criterion=criterion)


# Train and Evaluate DistilBERT
print("Training and evaluating DistilBERT...")
train_and_evaluate(distilbert_model, distilbert_train_loader, distilbert_eval_loader, device,optimizer=distilbert_optimizer,criterion=criterion)




Training and evaluating BERT...
Let's use 3 GPUs!


/home/brian/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1 completed, Average Training Loss: 1.9047
Epoch 2 completed, Average Training Loss: 1.2722
Precision: 0.5497, Recall: 0.5539, MCC: 0.5253, Log Loss: 1.3679, Hamming Loss: 0.4461
Training and evaluating RoBERTa...
Let's use 3 GPUs!


/home/brian/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1 completed, Average Training Loss: 1.9201
Epoch 2 completed, Average Training Loss: 1.3905
Precision: 0.5425, Recall: 0.5379, MCC: 0.5075, Log Loss: 1.4215, Hamming Loss: 0.4621
Training and evaluating XLNet...
Let's use 3 GPUs!


/home/brian/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1 completed, Average Training Loss: 1.9206
Epoch 2 completed, Average Training Loss: 1.3391
Precision: 0.5510, Recall: 0.5017, MCC: 0.4721, Log Loss: 1.4813, Hamming Loss: 0.4983
Training and evaluating DistilBERT...
Let's use 3 GPUs!


/home/brian/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1 completed, Average Training Loss: 1.8812
Epoch 2 completed, Average Training Loss: 1.2547
Precision: 0.5685, Recall: 0.5513, MCC: 0.5227, Log Loss: 1.3659, Hamming Loss: 0.4487
